In [1]:
# Packages

# To make data frames
import pandas as pd

# Numerical calculations
import numpy as np

# Timer
import time # for debugging

# For date arithmetic
from datetime import datetime, timedelta

In [2]:
# Functions

# Get how long a function takes (needs time)
def time_elapsed(start, end):
    total_time_seconds = end-start
    print("Total time elapsed:") 
    if total_time_seconds < 60:
        print(total_time_seconds, "seconds")
    else:
        total_time_minutes = total_time_seconds/60 
        if total_time_minutes < 60: 
            print(total_time_minutes, "minutes") 
        else: 
            total_time_hours = total_time_minutes/60 
            print(total_time_hours, "hours")
    print() 
    
# Add a row to df with given end_date, candidate_name, and party (needs df)
def insert_row(date, candidate, pop):
    global df
    df.at[len(df), 'end_date'] = date
    #df['end_date'][len(df-1)] = pd.to_datetime(df['end_date'][len(df-1)]).dt.date()
    df.at[len(df)-1, 'candidate_name'] = candidate
    df.at[len(df)-1, 'population'] = pop

# Impute a missing date for a candidate (needs the df)
def impute_for_candidate(date, candidate, pop):
    print("date, candidate, pop are "+str(date)+", "+candidate+", "+pop) # for debugging
    if ((df['end_date'] == date) & (df['candidate_name'] == candidate) & (df['population'] == pop)).any() == False:   
        print("No entry for this date, candidate, population.") # for debugging
        insert_row(date, candidate, pop)    
        print("Now we've inserted a row.") # for debugging
    else: # for debugging
        print("Candidate has an entry here.") # for debugging
    print() # for debugging

In [ ]:
# Read in the data

old_data = pd.read_csv("president_polls_cleaned.csv")
updated_data = pd.read_csv("president_polls.csv")

# Parse the dates

old_data['end_date'] = pd.to_datetime(old_data['end_date'], format='mixed').dt.date
updated_data['end_date'] = pd.to_datetime(updated_data['end_date'], format='mixed').dt.date

In [4]:
# Get the new data, along with the last week of old data

last_date = old_data['end_date'][0]

In [ ]:
df = updated_data[updated_data['end_date'] > last_date]
df['end_date']

In [6]:
# Restrict to national and swing state polls

swing_states = ["Arizona", "Georgia", "Michigan", "Nevada", "North Carolina", "Pennsylvania", "Wisconsin"]
df = df.loc[df['state'].isin(swing_states) | df['state'].isnull()]

In [7]:
# Delete superfluous columns

df.drop(columns=['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'display_name', 'pollster_rating_id', 
                 'sponsor_candidate_id', 'sponsor_candidate', 'sponsor_candidate_party', 'endorsed_candidate_id', 
                 'endorsed_candidate_name', 'endorsed_candidate_party', 'subpopulation', 'population_full', 'tracking', 
                 'created_at', 'notes', 'url', 'source', 'internal', 'race_id', 'cycle', 'office_type', 'seat_number', 
                 'seat_name', 'election_date', 'stage', 'nationwide_batch', 'ranked_choice_reallocated', 
                 'ranked_choice_round', 'answer', 'candidate_id'], inplace=True)

In [ ]:
# Replace the 'v' in the population column with 'rv'

df['population'].replace('v', 'rv', inplace=True)

In [9]:
# Use groupby to get daily averages according to population polled

cand_avg_by_date = df.groupby(['end_date', 'candidate_name', 'population'])['pct'].mean().reset_index(name='da_by_pop')
df = pd.merge(df, cand_avg_by_date)

In [10]:
# Get lists for imputing 

all_dates = df['end_date'].drop_duplicates() #pd.date_range(df['end_date'].tolist()[-1], df['end_date'].tolist()[0], freq='D')
candidates = ['Donald Trump', 'Joe Biden', 'Kamala Harris'] 
pops = ['lv', 'rv', 'a']

In [ ]:
all_dates

In [ ]:
# Impute the missing dates, candidate, populations 

start_impute = time.time()
for date in all_dates:
    for candidate in candidates:
        for pop in pops:
            impute_for_candidate(date, candidate, pop)
end_impute = time.time()
time_elapsed(start_impute, end_impute)   

In [13]:
# Reparse the dates, then take only the crucial data

#df['end_date'] = pd.to_datetime(df['end_date'], format='mixed').dt.date
crucial_data = df.loc[(df['candidate_name'] == 'Donald Trump') | (df['candidate_name'] == 'Joe Biden') | 
       (df['candidate_name'] == 'Kamala Harris'), ['population', 'candidate_name', 
                                                   'end_date', 'da_by_pop']].drop_duplicates()


In [ ]:
crucial_data.sort_values(['end_date', 'candidate_name', 'population']).tail(50)

In [ ]:
# Add in the last six days of the old data for the rolling average

crucial_data = pd.concat([old_data[old_data['end_date'] > last_date-pd.Timedelta(days=7)], crucial_data], ignore_index=True)
crucial_data

In [ ]:
# Format populations

crucial_data['population'].replace(['a', 'rv', 'lv'], ['All adults', ' Registered voters', 'Likely voters'], inplace=True)
crucial_data

In [ ]:
# Sort, then format the empty entries
 
crucial_data.sort_values(['candidate_name', 'population', 'end_date'], inplace=True, ignore_index=True)
crucial_data = crucial_data.replace('', np.NaN)
crucial_data.head(50)

In [ ]:
# Forward fill the missing averages
 
crucial_data['da_by_pop'] = crucial_data.groupby(['candidate_name', 'population'])['da_by_pop'].transform(lambda x: x.ffill())
crucial_data.head(50)

In [19]:
# Add column for rolling averages
   
crucial_data['pop_raverage7'] = crucial_data.groupby(['candidate_name', 
        'population']).rolling(7, min_periods=1)['da_by_pop'].mean().reset_index(drop=True)  

In [ ]:
crucial_data.head(50)

In [21]:
# Sort again

crucial_data.sort_values(by=['candidate_name', 'population', 'end_date'], ascending=[True, True, False], inplace=True)

In [ ]:
# Remove the last six days from the old_data

old_data = old_data[old_data['end_date'] <= last_date-pd.Timedelta(days=7)]
old_data

In [23]:
# Combine new data with old data, then export to a file

all_data = pd.concat([old_data, crucial_data], ignore_index=True)
all_data.sort_values(by=['candidate_name', 'population', 'end_date'], ascending=[True, True, False], inplace=True)
all_data.to_csv("president_polls_cleaned.csv", index=False)